In [1]:
import glob
files = glob.glob("Fire_images/*.*")
ls_fire = []
for i in files:
#     print([i.split("/")[1], '0'])
    ls_fire.append(['Fire_images', i.split("/")[1], '1'])

In [2]:
import pandas as pd

df_fire = pd.DataFrame(ls_fire, columns=['Folder', 'filename', 'label'])

In [3]:
df_fire

,Folder,filename,label
0,Fire_images,dsc_01001.jpg,1
1,Fire_images,burning-charcoal-briquettes.jpg,1
2,Fire_images,Chimney-Fire3.jpg,1
3,Fire_images,canada-wildfire.jpg,1
4,Fire_images,Living_Room_Camp_Fire.jpg,1
...,...,...,...
105,Fire_images,437438_616553-20151008-sprinkler_D.jpg,1
106,Fire_images,1.jpg,1
107,Fire_images,images.jpg,1
108,Fire_images,mfs_fire_investigation_image_bedroom_with_cand...,1


In [4]:
len(df_fire)


110

In [5]:
from PIL import Image
from keras.preprocessing.image import img_to_array

img = Image.open('{}/{}'.format(df_fire['Folder'][0], df_fire['filename'][0]))
print(img.size)
image_red = img.resize((1024, 1024))
image = img_to_array(image_red)

(3872, 2592)


In [6]:
image

array([[[ 67.,  23.,   1.],
        [ 68.,  26.,   2.],
        [ 68.,  27.,   1.],
        ...,
        [ 81.,  37.,   9.],
        [ 80.,  35.,   5.],
        [ 82.,  37.,   4.]],

       [[ 69.,  25.,   1.],
        [ 68.,  26.,   1.],
        [ 67.,  25.,   0.],
        ...,
        [ 82.,  38.,  11.],
        [ 79.,  34.,   4.],
        [ 80.,  35.,   4.]],

       [[ 69.,  26.,   1.],
        [ 68.,  26.,   1.],
        [ 69.,  28.,   1.],
        ...,
        [ 83.,  38.,  11.],
        [ 78.,  34.,   4.],
        [ 80.,  36.,   4.]],

       ...,

       [[ 25.,  17.,  14.],
        [ 27.,  19.,  16.],
        [ 29.,  21.,  18.],
        ...,
        [116.,  76.,  40.],
        [117.,  74.,  29.],
        [114.,  70.,  18.]],

       [[ 28.,  20.,  17.],
        [ 27.,  19.,  16.],
        [ 30.,  23.,  19.],
        ...,
        [ 98.,  63.,  27.],
        [ 95.,  59.,  14.],
        [ 98.,  59.,  11.]],

       [[ 30.,  23.,  20.],
        [ 28.,  21.,  17.],
        [ 30.,  

In [13]:
import numpy as np
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from hyperas.distributions import choice, uniform
from hyperas import optim

#Split df into train and test based on label column
df_train = df_fire
df_test = []

def data_gen(df, batch_size):
    while True:
        x_batch = np.zeros((batch_size, 3, 1024, 1024))
        y_batch = np.zeros((batch_size, 1))
        for j in range(len(df/batch_size)):
            b = 0
            for m, k in zip(df['filename'].values[j*batch_size:(j+1)*batch_size], df['label'].values[j*batch_size:(j+1)*batch_size]):
                img = Image.open('{}/{}'.format(df['Folder'][b], m))
                image_red = img.resize((1024, 1024))
                x_batch[b] = img_to_array(image_red)
                y_batch[b] = k
                b += 1
            yield (x_batch, y_batch)


batch_size = 32
epochs = 10

model = Sequential()

    #adding layers
model.add(Dense(512, input_shape=(1024,)))
model.add(Activation('relu'))
    #let's us find the dropout rate
model.add(Dropout(0.2, input_shape=(1024,)))
model.add(Dropout(0.2, input_shape=(1024,)))

model.add(Dense(10))
model.add(Activation('softmax'))


model.compile(loss='categorical_crossentrophy', optimizer='adam')


model.fit_generator(generator=data_gen(df_train, batch_size=batch_size), steps_per_epoch=len(df_train) // batch_size, epochs=epochs)

TypeError: unsupported operand type(s) for /: 'str' and 'int'